<a href="https://colab.research.google.com/github/Rivero-Romero/Deep_Learning/blob/main/01_Explocacion%20de%20datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalar librería de Kaggle
!pip install kaggle

In [ ]:
# cargar credenciales de kaggle
from google.colab import files

uploaded = files.upload()

In [ ]:
# Prepararación del entorno de Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Descargar Dataset
!kaggle datasets download -d kaustubhb999/tomatoleaf

In [ ]:
!unzip tomatoleaf.zip

In [ ]:
#Explorar la estructura de los directorios
import os
import pandas as pd

train_dir = "/content/tomato/train"

classes_train = os.listdir(train_dir)

data = []
# Contar imágenes por clase
for clase in classes_train:
    clase_path = os.path.join(train_dir, clase)
    num_images = len(os.listdir(clase_path))
    data.append({'Clase': clase, 'Número de imágenes': num_images})

# Crear un DataFrame con pandas
df = pd.DataFrame(data)
# Mostrar el DataFrame
print(df)

df.to_csv("resumen_entrenamiento.csv", index=False)

In [ ]:
#Ver muestras de las imágenes
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# Crear el generador de datos de entrenamiento
datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = datagen.flow_from_directory(
    '/content/tomato/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Obtener los índices de las clases y asignarles etiquetas legibles
class_indices = train_generator.class_indices  # Diccionario: {clase: índice}
class_labels = {v: k for k, v in class_indices.items()}  # Invertir el diccionario

plt.figure(figsize=(20, 20))

# Conjunto para rastrear las clases ya visitadas
visited_classes = set()

# Iterar a través del generador para extraer imágenes y etiquetas
for images, labels in train_generator:
    for i in range(len(labels)):
        label_index = np.argmax(labels[i])
        if label_index not in visited_classes:
            # Mostrar la imagen con su respectiva etiqueta
            plt.subplot(5, 5, len(visited_classes) + 1)
            plt.imshow(images[i])
            plt.title(class_labels[label_index])
            plt.axis('off')
            visited_classes.add(label_index)
        if len(visited_classes) == len(class_labels):
            break
    if len(visited_classes) == len(class_labels):
        break

# Mostrar las imágenes con etiquetas
plt.tight_layout()
plt.show()




In [ ]:
#Distribución de las clases
from collections import Counter
import matplotlib.pyplot as plt

# Definir las rutas de las carpetas de entrenamiento y validación
train_dir = '/content/tomato/train'
val_dir = '/content/tomato/val'
# Contar imágenes por clase
train_counts = {clase: len(os.listdir(os.path.join(train_dir, clase))) for clase in classes_train}
val_counts = {clase: len(os.listdir(os.path.join(val_dir, clase))) for clase in classes_train}

# Mostrar la distribución
print("Distribución en entrenamiento:")
print(train_counts)

print("Distribución en validación:")
print(val_counts)

# Crear la gráfica
plt.figure(figsize=(12, 6))

# Graficar la distribución de imágenes en entrenamiento y validación
bar_width = 0.4
index = range(len(train_counts))

plt.bar(index, train_counts.values(), width=bar_width, color='blue', label='Entrenamiento', align='center')
plt.bar([i + bar_width for i in index], val_counts.values(), width=bar_width, color='orange', label='Validación', align='center', alpha=0.7)

# Etiquetas de los ejes y título
plt.xticks([i + bar_width / 2 for i in index], train_counts.keys(), rotation=45, ha="right")
plt.title("Distribución de clases en Entrenamiento y Validación")
plt.xlabel("Clases")
plt.ylabel("Número de imágenes")

# Agregar leyenda
plt.legend()

# Ajustar el layout y mostrar la gráfica
plt.tight_layout()
plt.show()




In [ ]:
#Analizar las dimensiones y formatos de las imágenes

from PIL import Image

def check_image_properties(image_dir):
    sizes = []
    formats = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            img_path = os.path.join(root, file)
            try:
                with Image.open(img_path) as img:
                    sizes.append(img.size)
                    formats.append(img.format)
            except Exception as e:
                print(f"Error con la imagen {img_path}: {e}")

    print("Tamaños de las imágenes:", Counter(sizes))
    print("Formatos de las imágenes:", Counter(formats))

# Revisar datos de entrenamiento
check_image_properties(train_dir)


In [ ]:
#Verificar aumentación de datos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             rescale=1./255)

# Seleccionar una imagen de ejemplo
img_path = os.path.join(example_class, example_images[0])
img = load_img(img_path, target_size=(128, 128))
img_array = img_to_array(img)
img_array = img_array.reshape((1,) + img_array.shape)

# Generar 5 imágenes aumentadas
plt.figure(figsize=(10, 10))
i = 0
for batch in datagen.flow(img_array, batch_size=1):
    plt.subplot(1, 5, i + 1)
    plt.imshow(batch[0])
    plt.axis('off')
    i += 1
    if i == 5:  # Generar solo 5 imágenes
        break
plt.show()


In [ ]:
#Estadística básica
# Número total de imágenes
num_train = sum(train_counts.values())
num_val = sum(val_counts.values())

print(f"Número total de imágenes de entrenamiento: {num_train}")
print(f"Número total de imágenes de validación: {num_val}")
print(f"Proporción entrenamiento-validación: {num_train / num_val:.2f}")
